<a href="https://colab.research.google.com/github/mhyubr/News_Sentiment_Analysis_With_Stacking_Ensemble-FINAL-GEMASTIK_XV_2022/blob/main/News_Sentiment_Analysis_With_Stacking_Ensemble_GEMASTIKXV_DaBes_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import random

C:\ProgramData\Anaconda3\envs\EnvTensorFlow\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
from sklearn.model_selection import train_test_split

not_features = ['id', 'sentimen_berita']
date_cols = ['tanggal']
cat_cols = [col for col in df.columns if df[col].nunique() < 52]
text_cols = [col for col in df.columns if col not in not_features + date_cols + cat_cols]

high_card = [col for col in cat_cols if df[col].nunique() > 5]
low_card = [col for col in cat_cols if df[col].nunique() <= 5 and col not in not_features]

X = df[cat_cols + text_cols]
y = df['sentimen_berita'].map({'Negatif':0, 'Netral':1, 'Positif':2})

X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42, stratify=y)

In [ ]:
print(low_card)

['sumber', 'sentimen_kutipan']


In [ ]:
y_val.head()

41458    2
15305    1
31977    2
50387    2
27652    2
Name: sentimen_berita, dtype: int64

In [ ]:
X_train.shape

(42311, 16)

In [ ]:
#make model
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn import set_config
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler


set_config(display='diagram')

oh_encoder = OneHotEncoder()
ordinal_encoder = OrdinalEncoder()

high_card_preprocessor = Pipeline(steps=[
        ("ordinal_encoder", ordinal_encoder),])

low_card_preprocessor = Pipeline(steps=[
        ("oh_encoder", oh_encoder),])

preprocessor = ColumnTransformer([
        ("high_cardinality", high_card_preprocessor, high_card),
        ("low_cardinality", low_card_preprocessor, low_card),])

xgb_params = {
    'tree_method' : 'hist',
    'learning_rate': 0.065733165999498,
    'n_estimators': 500,
    'max_depth': 8,
    'lambda': 0.008103395718032142,
    'alpha': 0.37923490995294745,
    'colsample_bytree': 0.5,
    'min_child_weight': 19,
    'subsample': 1.0,
    'random_state' : 420,
    'use_label_encoder':False}

xgb = XGBClassifier(**xgb_params)

xgb_steps = [('preprocess', preprocessor),
             ('regressor', xgb),]

xgb_pipeline = Pipeline(xgb_steps)
xgb_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('high_cardinality',
                                                  Pipeline(steps=[('ordinal_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['kodekat', 'kodesubkat',
                                                   'kategori', 'subkategori']),
                                                 ('low_cardinality',
                                                  Pipeline(steps=[('oh_encoder',
                                                                   OneHotEncoder())]),
                                                  ['sumber',
                                                   'sentimen_kutipan'])])),
                ('regressor',
                 XGBClassifier(alpha=0.37923490995294745, base_sc...
                               lambda=0.008103395718032142,
                               learning_rate=0.065733165999498,
                               max_delta_step=None, max_depth=8,
                               min_child_weight=19, missing=nan,
                               monotone_constraints=None, n_estimators=500,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=420, reg_alpha=None,
                               reg_lambda=None, scale_pos_weight=None,
                               subsample=1.0, tree_method='hist',
                               use_label_encoder=False,
                               validate_parameters=None, ...))])

In [ ]:
xgb_pipeline.fit(X_train, y_train)

[17:24:40] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('high_cardinality',
                                                  Pipeline(steps=[('ordinal_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['kodekat', 'kodesubkat',
                                                   'kategori', 'subkategori']),
                                                 ('low_cardinality',
                                                  Pipeline(steps=[('oh_encoder',
                                                                   OneHotEncoder())]),
                                                  ['sumber',
                                                   'sentimen_kutipan'])])),
                ('regressor',
                 XGBClassifier(alpha=0.37923490995294745, base_sc...
                               learning_rate=0.065733165999498,
                               max_delta_step=0, max_depth=8,
                               min_child_weight=19, missing=nan,
                               monotone_constraints='()', n_estimators=500,
                               n_jobs=12, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=420, reg_alpha=0.37923491,
                               reg_lambda=0.00810339581, scale_pos_weight=None,
                               subsample=1.0, tree_method='hist',
                               use_label_encoder=False, ...))])

In [ ]:
print(text_cols)

['katakunci', 'judul_berita', 'konten_berita', 'nama_tokoh', 'jabatan', 'organisasi', 'lokasi', 'alias', 'kutipan']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
# from mlxtend.feature_selection import ColumnSelector

sgd_clf = SGDClassifier(
    loss='modified_huber',
    penalty='l1',
    alpha=1e-4,
    max_iter=45000,
    tol=6e-6,
    shuffle=True,
    verbose=False,
    random_state=420,
    learning_rate='adaptive',
    eta0=0.04,
    n_iter_no_change=237,)

# tfidf_1 = TfidfVectorizer()
# tfidf_2 = TfidfVectorizer()
tfidf_3 = TfidfVectorizer()

# nlp_1 = ColumnTransformer([
#     ("tfidf", tfidf_1, "katakunci"),
# ])

# nlp_2 = ColumnTransformer([
#     ("tfidf", tfidf_2, "judul_berita"),
# ])

nlp_3 = ColumnTransformer([
    ("tfidf", tfidf_3, "konten_berita"),
])

nlp_preprocessor = FeatureUnion([
#     ('katakunci', nlp_1),
#     ('judul_berita', nlp_2),
    ('konten_berita', nlp_3),
])

sgd_steps = ([
    ('preprocess', nlp_preprocessor),
    ('regressor', sgd_clf),
])

sgd_pipeline = Pipeline(sgd_steps)

sgd_pipeline

Pipeline(steps=[('preprocess',
                 FeatureUnion(transformer_list=[('konten_berita',
                                                 ColumnTransformer(transformers=[('tfidf',
                                                                                  TfidfVectorizer(),
                                                                                  'konten_berita')]))])),
                ('regressor',
                 SGDClassifier(eta0=0.04, learning_rate='adaptive',
                               loss='modified_huber', max_iter=45000,
                               n_iter_no_change=237, penalty='l1',
                               random_state=420, tol=6e-06, verbose=False))])

In [ ]:
from sklearn.neural_network import MLPClassifier
from category_encoders import TargetEncoder

target_encoder = TargetEncoder()

high_card_mlp_prep = Pipeline(steps=[
        ("target_encoder", target_encoder)])

low_card_mlp_prep = Pipeline(steps=[
        ("oh_encoder", oh_encoder),])

mlp_preprocessor = ColumnTransformer([
        ("high_cardinality", high_card_mlp_prep, high_card),
        ("low_cardinality", low_card_mlp_prep, low_card),])

mlp = MLPClassifier(
    hidden_layer_sizes = (256,2),
    activation = 'relu',
    solver = 'adam',
    alpha = 0.0002,
    batch_size = 'auto',
    learning_rate = 'adaptive',
    learning_rate_init = 0.001,
    max_iter = 2000,
    shuffle = True,
    random_state = 420,
    tol = 1e-8,
    verbose = False,)

mlpr_steps = [('preprocess', mlp_preprocessor),
             ('regressor', mlp),]
mlp_pipeline = Pipeline(mlpr_steps)

C:\ProgramData\Anaconda3\envs\EnvTensorFlow\lib\site-packages\category_encoders\target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
C:\ProgramData\Anaconda3\envs\EnvTensorFlow\lib\site-packages\category_encoders\target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

l2_model = LogisticRegression()

l1_models = [
    ("xgb", xgb_pipeline),
    ("mlpr", mlp_pipeline),
    ('sgd', sgd_pipeline),
    #('lgbm', lgbm_pipeline),
]

stacking_clf = StackingClassifier(estimators=l1_models,
                              final_estimator=l2_model,
                              cv=3,
                              stack_method='predict',
                              n_jobs=-1,
                              passthrough=False,
                              verbose=1)
stacking_clf

StackingClassifier(cv=3,
                   estimators=[('xgb',
                                Pipeline(steps=[('preprocess',
                                                 ColumnTransformer(transformers=[('high_cardinality',
                                                                                  Pipeline(steps=[('ordinal_encoder',
                                                                                                   OrdinalEncoder())]),
                                                                                  ['kodekat',
                                                                                   'kodesubkat',
                                                                                   'kategori',
                                                                                   'subkategori']),
                                                                                 ('low_cardinality',
                                                                                  Pipeline(steps=[('oh_encoder',
                                                                                                   OneHotEncoder())]),
                                                                                  ['sumber',
                                                                                   'sentimen_kutipan'])])),
                                                ('regressor',
                                                 XGBC...
                                                 FeatureUnion(transformer_list=[('konten_berita',
                                                                                 ColumnTransformer(transformers=[('tfidf',
                                                                                                                  TfidfVectorizer(),
                                                                                                                  'konten_berita')]))])),
                                                ('regressor',
                                                 SGDClassifier(eta0=0.04,
                                                               learning_rate='adaptive',
                                                               loss='modified_huber',
                                                               max_iter=45000,
                                                               n_iter_no_change=237,
                                                               penalty='l1',
                                                               random_state=420,
                                                               tol=6e-06,
                                                               verbose=False))]))],
                   final_estimator=LogisticRegression(), n_jobs=-1,
                   stack_method='predict', verbose=1)

In [ ]:
stacking_clf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\envs\EnvTensorFlow\lib\site-packages\category_encoders\target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
C:\ProgramData\Anaconda3\envs\EnvTensorFlow\lib\site-packages\category_encoders\target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
C:\ProgramData\Anaconda3\envs\EnvTensorFlow\lib\site-packages\category_encoders\target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
C:\ProgramData\Anaconda3\envs\EnvTensorFlow\lib\site-packages\category_encoders\target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.S

StackingClassifier(cv=3,
                   estimators=[('xgb',
                                Pipeline(steps=[('preprocess',
                                                 ColumnTransformer(transformers=[('high_cardinality',
                                                                                  Pipeline(steps=[('ordinal_encoder',
                                                                                                   OrdinalEncoder())]),
                                                                                  ['kodekat',
                                                                                   'kodesubkat',
                                                                                   'kategori',
                                                                                   'subkategori']),
                                                                                 ('low_cardinality',
                                                                                  Pipeline(steps=[('oh_encoder',
                                                                                                   OneHotEncoder())]),
                                                                                  ['sumber',
                                                                                   'sentimen_kutipan'])])),
                                                ('regressor',
                                                 XGBC...
                                                 FeatureUnion(transformer_list=[('konten_berita',
                                                                                 ColumnTransformer(transformers=[('tfidf',
                                                                                                                  TfidfVectorizer(),
                                                                                                                  'konten_berita')]))])),
                                                ('regressor',
                                                 SGDClassifier(eta0=0.04,
                                                               learning_rate='adaptive',
                                                               loss='modified_huber',
                                                               max_iter=45000,
                                                               n_iter_no_change=237,
                                                               penalty='l1',
                                                               random_state=420,
                                                               tol=6e-06,
                                                               verbose=False))]))],
                   final_estimator=LogisticRegression(), n_jobs=-1,
                   stack_method='predict', verbose=1)

In [ ]:
from sklearn.metrics import classification_report
val_preds = stacking_clf.predict(X_val)

print(classification_report(y_val, val_preds))

              precision    recall  f1-score   support

           0       0.61      0.51      0.56      1648
           1       0.56      0.44      0.49      3396
           2       0.74      0.87      0.80      5534

    accuracy                           0.68     10578
   macro avg       0.64      0.61      0.62     10578
weighted avg       0.66      0.68      0.66     10578

